# Shoreline Monitor High Resolution

Hotspots + monthly temporal resolution

Notebook environment to migrate netcdf files to CF compliant zarr

In [2]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [3]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_20996\2707390607.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [4]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("02_Shorelinemonitor_monthly")
dataset_dir_shorelinemonitor_hr = dataset_dir.joinpath("shorelinemonitor_monthly.nc")
dataset_out_file = "ShorelineMonitor_HR"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [5]:
# # # write csv to netcdf

# # # Load data from CSV file into a pandas dataframe
# csv_dir = r'P:\11202268-hydraulic-engineering\MSc_students\Dante_van_der_Heijden\02. Data\shorelineMonitor_highres.csv'
# df = pd.read_csv(csv_dir)

# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir_shorelinemonitor_hr)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [6]:
# open datasets
dataset_shorelinemonitor_hr = xr.open_dataset(dataset_dir_shorelinemonitor_hr)

# check original dataset
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
Data variables:
    Unnamed: 0         (index) int64 ...
    transect_id        (index) object ...
    transect_origin_x  (index) float64 ...
    transect_origin_y  (index) object ...
    transect_end_x     (index) float64 ...
    transect_end_y     (index) float64 ...
    dt                 (index) object ...
    dist               (index) object ...
    outliers           (index) object ...
    hotspot            (index) object ...
    Sdate              (index) object ...

<IPython.core.display.Javascript object>

In [7]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_shorelinemonitor_hr, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [8]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_shorelinemonitor_hr, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\02_Shorelinemonitor_monthly\\shorelinemonitor_monthly.nc': {'warnings': '22', 'errors': '7'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [9]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\02_Shorelinemonitor_monthly\metadata_shorelinemonitor_hr.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    dataset_shorelinemonitor_hr.attrs[attr_name] = attr_val

dataset_shorelinemonitor_hr.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [10]:
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop('Unnamed: 0')

<IPython.core.display.Javascript object>

In [11]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
Data variables:
    transect_id        (index) object ...
    transect_origin_x  (index) float64 ...
    transect_origin_y  (index) object ...
    transect_end_x     (index) float64 ...
    transect_end_y     (index) float64 ...
    dt                 (index) object ...
    dist               (index) object ...
    outliers           (index) object ...
    hotspot            (index) object ...
    Sdate              (index) object ...
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [12]:
from datetime import datetime

def round_to_nearest_month(date):
    # Extract year and month from the input date
    year = date.year
    month = date.month

    # Round the day to the nearest month
    if date.day > 15:
        month += 1
        if month > 12:
            year += 1
            month = 1
    else:
        month = month

    # Create a new date object for the rounded date
    rounded_date = datetime(year, month, 1)

    return pd.to_datetime(rounded_date)

<IPython.core.display.Javascript object>

In [13]:
max_dt = pd.to_datetime('1984-01-01')

for i, x in enumerate(dataset_shorelinemonitor_hr['dt'].values):
    try:    
        lst = json.loads(x)
        if len(lst) > 0:
            max_v = int(max(lst))
            end_date = round_to_nearest_month(round_to_nearest_month(pd.to_datetime(dataset_shorelinemonitor_hr['Sdate'].values[i])) + pd.DateOffset(days = max_v))
            if end_date > max_dt:
                max_dt = end_date
    except: 
        continue


<IPython.core.display.Javascript object>

In [14]:
sdate = pd.to_datetime(min(dataset_shorelinemonitor_hr['Sdate'].values))
#edate = sdate + pd.DateOffset(max_dt)
edate = max_dt

sdate, edate

(Timestamp('1984-04-19 00:00:00'), Timestamp('2021-12-01 00:00:00'))

<IPython.core.display.Javascript object>

In [15]:
# drop time xarray

# dataset_shorelinemonitor = dataset_shorelinemonitor.drop_vars('dt')
# TODO: this needs to be changes as start_dates are different now and not always at 1984-01

date_range = pd.date_range(start = sdate, end = edate, freq = 'MS')
dataset_shorelinemonitor_hr['time_step'] = xr.DataArray(date_range, dims = ['time']) #TODO: altered time_range into time_step
date_range

DatetimeIndex(['1984-05-01', '1984-06-01', '1984-07-01', '1984-08-01',
               '1984-09-01', '1984-10-01', '1984-11-01', '1984-12-01',
               '1985-01-01', '1985-02-01',
               ...
               '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
               '2021-11-01', '2021-12-01'],
              dtype='datetime64[ns]', length=452, freq='MS')

<IPython.core.display.Javascript object>

In [16]:
start_lats = []
for x in dataset_shorelinemonitor_hr['transect_origin_y'].values:
    try:
        x2 = float(x)
    except:
        x2 = float(str(x)[:-2])
    finally:
        start_lats.append(x2)


<IPython.core.display.Javascript object>

In [17]:
#combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = dataset_shorelinemonitor_hr['transect_origin_x'].values
end_lons = dataset_shorelinemonitor_hr['transect_end_x'].values
end_lats = dataset_shorelinemonitor_hr['transect_end_y'].values
coords = zip(zip(start_lons,start_lats), zip(end_lons,end_lats))

dataset_shorelinemonitor_hr['transect_geom'] = (['index'], [str(LineString(line)) for line in coords])
dataset_shorelinemonitor_hr['transect_geom'].attrs['long_name'] = 'Transect Geometry'

<IPython.core.display.Javascript object>

In [18]:
for i in range(len(dataset_shorelinemonitor_hr['dist'].values)):
    try:
        json.loads(dataset_shorelinemonitor_hr['dist'].values[i])
    except:
        print('hi', i, dataset_shorelinemonitor_hr['dist'].values[i], dataset_shorelinemonitor_hr['dt'].values[i])

hi 3940 [-5.010699810487949, -1.0004842285679842, -5.010699810487949, -8.387039460346159, 9.852722421875796, 9.852722421875796, 5.0572459455966055, 5.036067960786568, -1.0004842285679842, -1.0004842285679842, -1.0004842285679842, 1.3058702800924493, 1.3058702800924493, 5.036067960786568, 2.7383841312896493, -0.22185519951040078, 5.0572459455966055, -20.661152617808966, -1.0004842285679842, -1.0004842285679842, -2.1926713582427055, 5.036067960786568, 5.036067960786568, 5.036067960786568, 0.2959617024025647, 1.3058702800924493, -33.920871655376914, -0.9927849821381187, 0.1701167002006514, 5.036067960786568, 5.036067960786568, -5.010699810487949, -5.010699810487949, 5.036067960786568, 4.15168742143328, -5.085312938900643, -5.085312938900643, -21.24250283568631, -1.8812958932560377, -1.8812958932560377, -5.010699810487949, 5.036067960786568, 0.1701167002006514, 5.036067960786568, -5.010699810487949, -1.0004842285679842, 4.025442523043921, -8.46578048512265, 30.302956851782483, -26.69851734

<IPython.core.display.Javascript object>

In [ ]:
#TODO: Apply again after correct timesteps

def create_shorelinepos(time_range, positions, timesteps, sdates):
    time_range = [pd.to_datetime(date_str) for date_str in time_range]
    l = np.full((len(positions), len(time_range)), np.nan)

    for i in range(len(sdates)):

        try:
            positions_lst, timesteps_lst = np.array(json.loads(positions[i])), np.array(json.loads(timesteps[i]))
        except:
            positions_lst, timesteps_lst = [], []

        sdate = round_to_nearest_month(pd.to_datetime(sdates[i]))
        dates = [round_to_nearest_month(x) for x in pd.to_datetime(sdate) + pd.to_timedelta(timesteps_lst, unit='D')]
        mask = np.isin(time_range, dates)
        indices = np.arange(len(time_range))[mask]

        l[i, indices] = positions_lst

    return l

new_var1 = xr.apply_ufunc(create_shorelinepos, 
                          dataset_shorelinemonitor_hr['time_step'].values, 
                          dataset_shorelinemonitor_hr['dist'].values, 
                          dataset_shorelinemonitor_hr['dt'].values, 
                          dataset_shorelinemonitor_hr['Sdate'].values,
                          input_core_dims=[['time_step'], ['index'], ['index'], ['index']], 
                          dask='parallelized',
                          output_dtypes=[float], 
                          output_core_dims=[['time_step']],
                          vectorize=False)

<IPython.core.display.Javascript object>

In [ ]:
new_var1

NameError: name 'new_var1' is not defined

In [ ]:
def combine_outliers(positions, outl):
    l = np.zeros(len(positions))
    not_na = np.where(~np.isnan(positions))[0]
    outliers = np.array(json.loads(outl))
    l[not_na[outliers]] = 1

    return l

new_var2 = xr.apply_ufunc(
    combine_outliers,
    new_var1,
    dataset_shorelinemonitor_hr['outliers'],
    input_core_dims=[['time_step'], []],
    output_core_dims=[['time_step']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[np.float32]
)

IndexError: arrays used as indices must be of integer (or boolean) type

<IPython.core.display.Javascript object>

In [ ]:
delete_vars = ['dist', 'outliers']
for dv in delete_vars:
    dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop(dv)

dataset_shorelinemonitor_hr['dist'] = (['index', 'time'], new_var1.data)
dataset_shorelinemonitor_hr['outliers'] = (['index', 'time'], new_var2.data)

In [ ]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\02_Shorelinemonitor_monthly\vars_shorelinemonitor_hr.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            dataset_shorelinemonitor_hr[var_dict['name']].attrs[key] = value

delete_vars = ['transect_end_x', 'transect_end_y']
for dv in delete_vars:
    dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop(dv)
    
# change dtypes
object_vars = ['transect_id', 'hotspot_id']
for ov in object_vars:
    dataset_shorelinemonitor_hr[ov] = dataset_shorelinemonitor_hr[ov].astype('S')

all_vars = list(dataset_shorelinemonitor_hr.keys())
data_vars = ['sp', 'outliers']
# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.set_coords([v for v in all_vars if v not in data_vars])

# drop index xarray
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop('index')

<IPython.core.display.Javascript object>

In [ ]:
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop(['times', 'time_step'])

<IPython.core.display.Javascript object>

In [ ]:
#dataset_shorelinemonitor = dataset_shorelinemonitor.rename_vars({'time_step' : 'time'})
rename = {"long_name": "Time", "standard_name": "time", "units": "yr"}

dataset_shorelinemonitor_hr['time'].attrs['long_name'] = rename['long_name']
dataset_shorelinemonitor_hr['time'].attrs['standard_name'] = rename['standard_name']
dataset_shorelinemonitor_hr['time'].attrs['units'] = rename['units']

<IPython.core.display.Javascript object>

In [ ]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/18)
    transect_id         (stations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    country_id          (stations) |S7 b'CHL' b'CHL' b'CHL' ... b'RUS' b'RUS'
    continent           (stations) |S23 b'South America' ... b'Europe'
    country             (stations) |S40 b'Chile' b'Chile' ... b'Russia'
    no_shorelines       (stations) float64 ...
    rmse                (stations) float64 ...
    ...                  ...
    no_sedcomp          (stations) float64 ...
    low_detect_shlined  (stations) float64 ...
    err_changerate      (stations) float64 ...
    err_timespan        (stations) float64 ...
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    transect_geom       (stations) |S92 b'LINESTRING (-74.3863095545 -50.3776...
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 ...
    changerate_unc      (stations) float64 ...
    sandy               (stations) bool ...
    sp                  (stations, time) float64 nan nan nan ... nan 587.6 nan
    outliers            (stations, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    SHORT_DESCRIPTION:   The Shoreline Monitor is a global dataset that provi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41598-018-24630-6
    LONG_NAME:           THE_SHORELINE_MONITOR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [ ]:
# save new .nc files
dataset_shorelinemonitor_hr['time'].attrs.pop('units', None)
dataset_shorelinemonitor_hr.to_netcdf(path=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\01_Shorelinemonitor_annual\\shorelinemonitor_annual_CF.nc': {'warnings': '4', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [ ]:
dataset_shorelinemonitor['outliers'].values[0]

In [ ]:
# export to zarr in write mode (to overwrite if exists)
dataset_shorelinemonitor.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [ ]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>